## Árvore de Decisão — Câncer de Mama

## 0. Imports

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, roc_curve, classification_report
)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


## 1. Dados

In [ ]:

data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names) # cada linha de x é um nódulo e cada coluna é uma medida desse nódulo
y = pd.Series(data.target, name="target")  # guarda que tipo de nódulo é, 0=maligno, 1=benigno

print(f"Formato X: {X.shape}, y: {y.shape}")
display(X.head())


class_counts = y.value_counts().sort_index()
print("\nContagem por classe (0=maligno, 1=benigno):")
print(class_counts)

plt.figure()
plt.bar(class_counts.index.astype(str), class_counts.values)
plt.title("Distribuição de classes (0=maligno, 1=benigno)")
plt.xlabel("Classe"); plt.ylabel("Contagem")
plt.show()


## 2. Divisão treino/teste (70/30)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split( # divide 70% dos dados para treinar a árvore e o restante pra testar sua performance
    X, y,
    test_size=0.30, #30% para teste da árvore
    random_state=RANDOM_STATE,
    stratify=y
)
print(f"Treino: {X_train.shape}, Teste: {X_test.shape}")


## 3. Modelo base: Árvore de Decisão

In [ ]:

clf = DecisionTreeClassifier(random_state=RANDOM_STATE) # a árvore é criada
clf.fit(X_train, y_train) # a árvore é treinada com os dados passados anteriormente
# depois do treino, a árvore será testada e mostrará os parâmetros gerados
print("Profundidade da árvore:", clf.get_depth()) # profundidade da árvore, ou seja, o número máximo de perguntas que a árvore faz para chegar a uma decisão
print("Número de folhas:", clf.get_n_leaves()) # número de folhas, ou seja, o número de decisões finais que a árvore pode tomar


## 3. Avaliação no teste — métricas principais

In [ ]:

y_pred = clf.predict(X_test)
# com base nos testes que ela fez, as métricas (taxas de desempenho) são exibidas
acc = accuracy_score(y_test, y_pred) 
prec = precision_score(y_test, y_pred, zero_division=0)
rec = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

print(f"Acurácia : {acc:.4f}") # de x casos, ela acerta quantos? acertos no geral
# precisão: de todos os casos que ela disse que era positivo, quantos realmente eram positivos? baixa precisão = muitos falsos positivos
# recall: de todos os casos que eram positivos, quantos ela conseguiu identificar? baixa revocação = muitos falsos negativos
# f1: média harmônica para medir equilíbrio entre precisão e revocação 
# 0 = maligno, 1 = benigno

print("\nRelatório de classificação:") # 
from sklearn.metrics import classification_report

rep = classification_report(
    y_test, y_pred,
    output_dict=True,           # devolve um dicionário (não o texto pronto)
    zero_division=0
)

import pandas as pd
df = pd.DataFrame(rep).T

df = df.loc[["0", "1"], ["precision", "recall", "f1-score", "support"]]
print(df.to_string())



## 4. Visualização da árvore (modelo base)

In [ ]:

plt.figure(figsize=(18, 10))
plot_tree(
    clf,
    feature_names=X.columns,
    class_names=["maligno (0)", "benigno (1)"],
    filled=True,
    rounded=True
)
plt.title("Árvore de Decisão — Modelo base")
plt.show()


## 5. Importância dos atributos (Top-10)

In [ ]:

importances = pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False)
topk = 10
top_imp = importances.head(topk)

# Soma de todas as reduções de impureza que esse atributo causou ao longo da árvore, 
# ponderadas pelo nº de amostras que passaram pelo nó, e normalizada para somar 1.0 no final.
# Resumindo, quanto mais o atributo diminui a impureza de Gini, mais importante ele é.

display(top_imp.to_frame("importance"))

plt.figure()
plt.barh(top_imp.index[::-1], top_imp.values[::-1])
plt.title(f"Top {topk} atributos mais importantes (modelo base)")
plt.xlabel("Importância (Gini)"); plt.ylabel("Atributo")
plt.tight_layout()
plt.show()


## 6. Controle de complexidade - Poda por custo 

In [ ]:

path = clf.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas

accs = []
models = []

for alpha in ccp_alphas:
    m = DecisionTreeClassifier(random_state=RANDOM_STATE, ccp_alpha=alpha)
    m.fit(X_train, y_train)
    y_pred_m = m.predict(X_test)
    acc_m = accuracy_score(y_test, y_pred_m)
    accs.append(acc_m)
    models.append(m)

best_idx = int(np.argmax(accs))
best_alpha = ccp_alphas[best_idx]
best_acc = accs[best_idx]

print(f"Melhor alpha (teste) = {best_alpha:.6f} | Acurácia = {best_acc:.4f}")

plt.figure()
plt.plot(ccp_alphas, accs, marker="o")
plt.title("Acurácia no teste vs ccp_alpha (poda)")
plt.xlabel("ccp_alpha")
plt.ylabel("Acurácia (teste)")
plt.show()

clf_pruned = models[best_idx]
print("Profundidade (podada):", clf_pruned.get_depth(), " | Folhas:", clf_pruned.get_n_leaves())

# Os alphas gerados vão gradualmente podar as ramificações menos importantes da árvore, até chegar em um valor ótimo.

### 7.1 Comparação: modelo base vs. podado (métricas principais)

In [ ]:

def metrics_report(model, Xte, yte, name="model"):
    ypred = model.predict(Xte)
    acc = accuracy_score(yte, ypred)
    prec = precision_score(yte, ypred, zero_division=0)
    rec = recall_score(yte, ypred, zero_division=0)
    f1 = f1_score(yte, ypred, zero_division=0)
    print(f"[{name}] acc={acc:.4f} | prec={prec:.4f} | rec={rec:.4f} | f1={f1:.4f}")
    return {"acc": acc, "prec": prec, "rec": rec, "f1": f1}

m_base = metrics_report(clf, X_test, y_test, "base")
m_pruned = metrics_report(clf_pruned, X_test, y_test, "podado")


### 7.2 Árvore podada (visual)

In [ ]:

plt.figure(figsize=(18, 10))
plot_tree(
    clf_pruned,
    feature_names=X.columns,
    class_names=["maligno (0)", "benigno (1)"],
    filled=True,
    rounded=True
)
plt.title("Árvore de Decisão — Modelo podado")
plt.show()
